In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import seaborn as sns 

import assessment 
import calibration 
import utils 
from utils import logit, sigmoid
import warnings 
warnings.filterwarnings("default")

%load_ext autoreload
%matplotlib inline
%autoreload 2

mpl.rcParams.update({'font.size': 15})
mpl.rcParams.update({"axes.grid" : True, "grid.linestyle": '--', 
                     "grid.alpha": 0.8, "grid.color": "black"})
mpl.rcParams.update({"lines.linewidth" : 3})
mpl.style.use('seaborn-colorblind')

In [ ]:
n_train = 1000
n_val = 500
n_test = 4500

np.random.seed(0)
class true_model:
    def predict_proba(x):
        prob_1 = 0.1 + 0.8*np.mod(np.floor(x/5), 2)
        if(len(prob_1.shape)==1):
            return np.vstack((1-prob_1, prob_1)).T
        else: 
            return np.hstack((1-prob_1, prob_1))

def create_data(n):    
    dat = np.zeros((n, 1))
    Y = np.zeros((n))
    for i in range(n):
        dat[i,:] = 2*np.random.randn((1))
        dat[i,:] += (create_data.count/250)
        prob = true_model.predict_proba(dat[i,:])[:,1]
        Y[i] = int(np.random.random() <= prob)
        create_data.count += 1
    return dat, Y

create_data.count = 0

x_train, y_train = create_data(n_train)
x_calib, y_calib = create_data(n_val)
x_test, y_test = create_data(n_test)

In [ ]:
class trig_logistic:
    def __init__(self):
        self.model = LogisticRegression()#RandomForestClassifier(n_estimators=1000)
        self.translate = np.pi*np.array([0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75])
        self.freq = np.array([1, 2, 3, 4, 5, 6])
        self.g1, self.g2 = np.meshgrid(self.translate, self.freq)
    def create_features(self, x):
        x_feats = np.zeros((x.shape[0], self.g1.size))
        for i, el in enumerate(zip(self.g1.flatten(), self.g2.flatten())):
            x_feats[:,i] = np.sin((x[:,0]+(el[0]*el[1]))/el[1])
        return x_feats
    def fit(self, x, y):
        x_feats = self.create_features(x)
        self.model.fit(x_feats, y)
        return self
    def predict_proba(self, x):
        x_feats = self.create_features(x)
        return self.model.predict_proba(x_feats)

In [ ]:
x = np.vstack((np.arange(0, 20, 0.05), np.arange(0, 20, 0.05))).T
x_feats = trig_logistic().create_features(x)
fig, ax = plt.subplots(1,1,figsize=(5,3),constrained_layout=True)
for i in range(8, 16):
    ax.plot(x[:,0] + x[:,1], x_feats[:,i], label = "Feat {}".format(i))
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
model = trig_logistic().fit(x_train, y_train)
_, a_platt, b_platt = calibration.fit_platt_scaling_parameters(logit(model.predict_proba(x_calib)[:,1]), y_calib)

In [ ]:
pred_probs_ons, a_platt_ons, b_platt_ons = calibration.online_platt_scaling_newton(logit(model.predict_proba(np.vstack((x_calib, x_test)))[:,1]), np.concatenate((y_calib, y_test)))

In [ ]:
plt.plot(a_platt_ons, label="a ONS")
plt.plot(b_platt_ons, label="b ONS")
plt.legend()
plt.title("a_fixed = {:.2}, b_fixed = {:.2}".format(a_platt[0], b_platt[0]))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

x_vals = (np.arange(-4, 30, 0.1)).reshape(-1,1)
plt.hist(x_train[:,0], weights=0.003*np.ones(1000), alpha=0.2, label="Training data")
plt.hist(x_test[2000:2500,0], weights=0.007*np.ones(500), alpha=0.2, label="Test data from t = 2500 to t = 3000")
plt.hist(x_test[4000:,0], weights=0.007*np.ones(500), alpha=0.2, label="Test data from t = 4500 to t = 5000")
plt.plot([], [])
plt.plot(x_vals[:,0], true_model.predict_proba(x_vals)[:,1], label = "True model", linestyle='--')
plt.plot(x_vals[:,0], model.predict_proba(x_vals)[:,1], label = "Original model", linestyle='dotted')
plt.plot(x_vals[:,0], sigmoid(a_platt*logit(model.predict_proba(x_vals)[:,1]) + b_platt), label = "Fixed batch Platt scaling at t = 500", linestyle='dashdot')
plt.plot(x_vals[:,0], sigmoid(a_platt_ons[500]*logit(model.predict_proba(x_vals)[:,1]) + b_platt_ons[500]), label = "Online Platt scaling at t = 2500", linestyle='dashed')
plt.plot(x_vals[:,0], sigmoid(a_platt_ons[2500]*logit(model.predict_proba(x_vals)[:,1]) + b_platt_ons[2500]), label = "Online Platt scaling at t = 2500", linestyle='dashed')
plt.plot(x_vals[:,0], sigmoid(a_platt_ons[4500]*logit(model.predict_proba(x_vals)[:,1]) + b_platt_ons[4500]), label = "Online Platt scaling at t = 4500", linestyle=(0, (3, 2, 1, 2, 1, 2)))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xticks(fontsize=30); plt.yticks(fontsize=30);
ax.set_ylim([0,1]);

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
x_vals = np.arange(-5, 30, 0.1).reshape((-1,1))

true_probs = true_model.predict_proba(x_vals)[:,1]
ax[0,0].plot([], []) 
ax[0,0].plot([], []) 
ax[0,0].plot([], [])
ax[0,0].plot([], []) 
ax[0,0].plot(x_vals, true_probs, label = "True probabilities at t = 0", linestyle='solid') 
ax[0,0].plot([], [])
ax[0,0].plot(x_vals, model.predict_proba(x_vals)[:,1], label = "Fixed base model", linestyle='dotted')#, color=line.get_color())
utils.normalized_hist(x_train, ax[0,0])
ax[0,0].set_title(r'$t = 1$ to $t = 1000$ (training)')

t1 = 1500
true_probs = true_model.predict_proba(x_vals)[:,1]
ax[0,1].plot([], []) 
ax[0,1].plot([], []) 
ax[0,1].plot([], [])
ax[0,1].plot([], [])
ax[0,1].plot(x_vals, true_probs, label = "True probabilities at t = 1500", linestyle='solid')
ax[0,1].plot([], []) 
ax[0,1].plot(x_vals, model.predict_proba(x_vals)[:,1], label = "Fixed base model", linestyle='dotted')#, color=line.get_color())
ax[0,1].plot([], []) 
ax[0,1].plot(x_vals, sigmoid(a_platt_ons[t1-1000]*logit(model.predict_proba(x_vals)[:,1]) + b_platt_ons[t1-1000]), label = "Online Platt scaling (OPS) at t = 1500", linestyle='dashed')
utils.normalized_hist(x_test[t1-1500:t1-1000], ax[0,1])
ax[0,1].set_title(r'$t = 1501$ to $t = 2000$')

t1 = 3500
true_probs = true_model.predict_proba(x_vals)[:,1]
ax[1,0].plot([], []) 
ax[1,0].plot([], []) 
ax[1,0].plot([], [])
ax[1,0].plot([], [])
ax[1,0].plot(x_vals, true_probs, label = "True probabilities at t = 1500", linestyle='solid')
ax[1,0].plot([], []) 
ax[1,0].plot(x_vals, model.predict_proba(x_vals)[:,1], label = "Fixed base model", linestyle='dotted')#, color=line.get_color())
ax[1,0].plot([], []) 
ax[1,0].plot(x_vals, sigmoid(a_platt_ons[t1-1000]*logit(model.predict_proba(x_vals)[:,1]) + b_platt_ons[t1-1000]), label = "Online Platt scaling (OPS) at t = 1500", linestyle='dashed')
utils.normalized_hist(x_test[t1-1500:t1-1000], ax[1,0])
ax[1,0].set_title(r'$t = 3501$ to $t = 4000$')

t1 = 5500
true_probs = true_model.predict_proba(x_vals)[:,1]
ax[1,1].plot([], []) 
ax[1,1].plot([], []) 
ax[1,1].plot([], [])
ax[1,1].plot([], [])
ax[1,1].plot(x_vals, true_probs, label = "True probabilities at t = 1500", linestyle='solid')
ax[1,1].plot([], []) 
ax[1,1].plot(x_vals, model.predict_proba(x_vals)[:,1], label = "Fixed base model", linestyle='dotted')#, color=line.get_color())
ax[1,1].plot([], []) 
ax[1,1].plot(x_vals, sigmoid(a_platt_ons[t1-1000]*logit(model.predict_proba(x_vals)[:,1]) + b_platt_ons[t1-1000]), label = "Online Platt scaling (OPS) at t = 5500", linestyle='dashed')
utils.normalized_hist(x_test[t1-1500:t1-1000], ax[1,1])
ax[1,1].set_title(r'$t = 5501$ to $t = 6000$')

fig.add_subplot(111, frameon=False)
plt.grid(False)
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.xlabel(r'Value of $x$', fontsize=22)
plt.ylabel(r'True/predicted   Pr$(Y=1 \mid X = x)$', fontsize=20)

plt.savefig('results/covariate_shift_1d.pdf')

In [ ]:
def print_vals(y_pred_probs, true_probs):
    y_pred_classes = (y_pred_probs>=0.5).astype('int')
    ece = 0
    bin_edges = np.sort(np.unique(y_pred_probs))
    bin_assignment = utils.bin_points(y_pred_probs, bin_edges)
    tot_elem = 0
    for i, bin_edges in enumerate(bin_edges):
        bin_idx = (bin_assignment == i)
        assert(sum(bin_idx) > 0), "This assert should pass by construction of the code"
        n_elem = sum(bin_idx)
        tot_elem += n_elem
        pi_pred = y_pred_probs[bin_idx].mean()
        pi_true = true_probs[bin_idx].mean()
        ece += n_elem*np.abs(pi_pred-pi_true)
    assert(tot_elem == y_pred_probs.size)
    ece /= y_pred_probs.size
    acc = np.sum(np.multiply(true_probs, y_pred_classes) + np.multiply(1-true_probs,1-y_pred_classes))/y_pred_probs.size    

    print("{:.2f}\\% & {:.2}".format(100*acc, ece))

In [ ]:
y_pred_probs = model.predict_proba(x_train)[:,1]
true_probs = true_model.predict_proba(x_train)[:,1]
print_vals(y_pred_probs, true_probs)
for base_val in [0, 2000, 4000]:
    y_pred_probs = model.predict_proba(x_test)[base_val:(base_val+500),1]
    true_probs = true_model.predict_proba(x_test)[base_val:(base_val+500),1]
    print_vals(y_pred_probs, true_probs)
    y_pred_probs = sigmoid(np.multiply(a_platt_ons[(base_val + 500):(base_val + 1000)], 
                                       logit(model.predict_proba(x_test)[base_val:(base_val+500),1])) +
                           b_platt_ons[(base_val + 500):(base_val + 1000)])
    print_vals(y_pred_probs, true_probs)